In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, date

## CFM 101: Group Assignment - Python Roboadvisor
### Team Number: 15
### Team Member Names: Landon Trinh, Ethan Zemelman, Jessie Deng
### Team Strategy Chosen: SAFE

## Goal
- Our team has decided to target dynamically building the safest portfolio
- Given a file of unknown stock tickers, our roboadvisor will run from November 25, 2023 to December 4, 2023
- The deviation will be calculated by taking calculating the difference between the final value of the portfolio and the initial portfolio value ($750,000)
- Ultimately, our aim is to have our portfolio deviate in nominal value. In other words, in the end, we want our portfolio value to have changed as little as possible


## Introduction
> "Theory will only take you so far." - J. Robert Oppenheimer

In theory, our trading strategy should produce a portfolio that deviates in nominal value the least by taking into consideration the following:
- Beta
- Diversification
- Correlation
- Standard deviation
- Expected returns

Our goal will be to tell a convincing story to WHY we are picking our stocks. We will calculate and discuss statistics, display and intepret graphs, and explain our thought process

## 1. Setup
Before implementing our trading strategy, we will initialize required and useful constants as part of the rules:
- Currency of valid stocks (USD or CAD)
- Required average monthly volume (150,000 shares)
- The number of stocks we wish to purchase on the start date (10-22 stocks)
- Time interval (Janurary 1, 2023 - October 31, 2023)
- Minimum number of trading days for month (18 days)
- Minimum stock weighting: $\frac{100}{2n}$%, $n$ = number of stocks in portfolio
- Maximum stock weighting: 20%
- Initial investment amount: $750,000 CAD
- Buying date of roboadvisor: November 25, 2023 - December 4, 2023
- Trading fee for each stock trade: $4.95 CAD

In the end, our roboadvisor should create two DataFrames:

1. $\verb|Portfolio_Final|\\$
- Index: Starts at 1 and ends at number of stocks in portfolio
- Headings: Ticker, Price (price of stock on Nov 25), Currency (CAD or USD), Shares, Value, Weight (adds to 100%)

2. $\verb|Stocks_Final|\\$
We should output this DataFrame to a CSV file titled "Stocks_Group_15.csv"
- Index: Same as "Portfolio Final"
- Headings: Tickers and Shares from "Portfolio_Final"


In [2]:
# Investment amount (CAD)
capital = 750_000

# Number of stocks to buy for portfolio
num_stocks = 15

# Maximum and minimum weightings of each stock in portfolio
min_weight = 1 / (2 * num_stocks)
max_weight = 0.20

# Start and end date for roboadvisor
# start_date = "2023-11-25"
# end_date = "2023-12-04"

# Filtering requirements
valid_currency = ["CAD", "USD"]
min_trading_days = 18
required_avg_volume = 150000

## 2. Filtering
After reading in the CSV file containg stock tickers, we must filter the list of stocks to make sure they are valid stock tickers according to the following rules:
- Include stocks that have an average monthly volume of at leaest 150,000 shares based on Jan 1, 2023 - Oct 31, 2023 (drop any months that don't have at least 18 trading days)
- Stock denominated in USD or CAD

In [3]:
# Read in CSV ticker file
tickers = pd.read_csv("tickers_example.csv", header=None)
tickers = tickers.rename(columns={0: "ticker"})
tickers_lst = tickers["ticker"].tolist()
tickers.head()

,ticker
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AGN


In [4]:
# Set parameters for filtering tickers
filter_start_date = "2023-01-01"
filter_end_date = "2023-10-31"
filter_interval = "1mo"

In [5]:
# Determines months with less than 18 trading days
def get_short_months(market_index):
    short_months = []
    for month in range(1, 11):
        trading_days = len(market_index.history(start=str(date(2023, month, 1)), end=str(date(2023, month+1, 1))))
        if trading_days < min_trading_days:
            short_months.append(month)
    return short_months

# Keeps stocks with valid average monthly volume
def filter_volume(tickers, short_months):

    # Retrieve monthly volume data for tickers
    volume_data = yf.download(tickers=tickers, interval="1mo", start=filter_start_date, end=filter_end_date).Volume

    # Drop short months from volume DataFrame
    for short_month in short_months:
        volume_data.drop(str(date(2023, short_month, 1)))

    # Determine whether stocks meets average monthly volume requirement
    for ticker in tickers:
        if (volume_data[ticker]).mean() < min_avg_volume:
            print(f"{ticker} does not meet the required minimum average monthly volume")
            tickers.remove(ticker)

    # Return finalized list of tickers
    return tickers


# Retrieve filtered tickers
def filter_tickers(tickers):
    
    # Initialize list to separately store CAD and USD tickers
    cad_tickers = []
    usd_tickers = []
    
    for ticker in tickers:
        try:
            stock_ticker = yf.Ticker(ticker)
            base_currency = stock_ticker.fast_info["currency"]
            
            # Store ticker in appropriate list
            if base_currency == "CAD":
                cad_tickers.append(ticker)
            
            elif base_currency == "USD":
                usd_tickers.append(ticker)
    
        except:
            print(f"{ticker} may be delisted")

    # Determine months that have less than 18 trading days for CAD and USD stocks
    cad_short_months = get_short_months(yf.Ticker("^GSPTSE"))
    usd_short_months = get_short_months(yf.Ticker("^GSPC"))

    # Filter months that have an average monty volume of less than 150k
    filtered_cad_tickers = filter_volume(cad_tickers, cad_short_months)
    filtered_usd_tickers = filter_volume(usd_tickers, usd_short_months)

    # Add to single list
    filtered_tickers = filtered_cad_tickers + filtered_usd_tickers
    return filtered_tickers
    
filtered_tickers = filter_tickers(tickers_lst)
print(filtered_tickers)

In [6]:
# Download stock data
stock_data = yf.download(tickers=filtered_tickers, interval="1d", start=filter_start_date, end=filter_end_date).Close.dropna()
stock_data

AGN may be delisted
CELG may be delisted
MON may be delisted
RTN may be delisted


[*********************100%%**********************]  36 of 36 completed


,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-03,125.070000,162.380005,109.580002,270.260010,62.930000,85.820000,147.119995,195.389999,33.509998,272.630005,...,107.199997,128.029999,48.790001,26.320000,87.669998,163.210007,518.640015,207.580002,175.279999,44.639999
2023-01-04,126.360001,163.690002,111.209999,269.339996,63.860001,85.139999,150.539993,203.639999,34.139999,270.809998,...,111.529999,129.169998,50.610001,26.639999,88.809998,169.169998,504.500000,209.240005,177.110001,46.029999
2023-01-05,125.019997,163.490005,110.800003,262.980011,63.509998,83.120003,146.429993,204.990005,34.070000,271.589996,...,109.400002,128.789993,48.830002,26.610001,86.410004,166.929993,489.959991,203.080002,173.839996,45.669998
2023-01-06,129.619995,166.550003,112.330002,269.209991,64.550003,86.080002,150.169998,213.000000,34.410000,279.250000,...,115.339996,130.429993,49.560001,27.020000,86.370003,175.160004,490.000000,212.009995,178.949997,46.310001
2023-01-09,130.149994,161.660004,112.150002,273.750000,63.869999,87.360001,150.399994,208.570007,33.889999,274.720001,...,114.610001,131.399994,49.810001,26.959999,86.099998,176.679993,490.059998,211.460007,181.690002,46.610001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,173.440002,146.309998,94.809998,296.089996,59.869999,128.559998,144.419998,182.360001,25.469999,252.110001,...,109.389999,110.379997,71.870003,22.360001,76.830002,146.919998,525.000000,205.440002,149.320007,31.389999
2023-10-25,171.100006,145.259995,93.570000,292.679993,60.959999,121.389999,143.520004,177.729996,25.549999,246.720001,...,104.779999,109.110001,66.900002,22.280001,76.919998,141.789993,530.210022,205.220001,146.929993,31.290001
2023-10-26,166.889999,145.199997,93.980003,292.040009,60.849998,119.570000,143.339996,179.089996,26.120001,241.080002,...,105.620003,110.209999,64.529999,22.320000,77.410004,144.009995,528.359985,202.240005,138.210007,31.770000


## 3. Stock Analysis
In this step we will choose our stocks based on various measures.
- Get standard deviation of each stock
- Get the beta of each stock
- Get the expected returns of each stock
- Create a correlation matrix of all the stocks

In [8]:
std_dict = {}
betas = {}
exp_returns = {}

final_closings = pd.DataFrame()
Stocks_Final = pd.DataFrame()

## 4. Portfolio Optimization
- Create random weights for each stock, and create n number of random portfolios
- Choose the portfolio with the lowest expected returns

In [9]:
Portfolio_Final = pd.DataFrame()

def random_weights(n, min, max):
    """
    Generates a list of n number of random weights, each between min and max, that sum to 1
    """

    # TODO: Must modify this function to ensure final weightings are within min and max (currently after normalizing weights, they are not)
    # Create an array of n number of random weights that sum to 1
    weights = np.random.uniform(min, max, n)
    weights /= np.sum(weights)
    return weights

def random_portfolios(num_portfolios, closing_prices):
    """
    Generates a list of num_portfolios number of random portfolios (each stored in a dataframe) by randomly assigning weights to each stock

    Parameters:
    num_portfolios (int): Number of random portfolios to generate
    closing_prices (pd.DataFrame): Dataframe containing closing prices for each stock

    Returns:
    portfolios (dictionary): Dictionary containing the randomly generated portfolios.
                       Each portfolio is a dataframe containing the stocks' daily values in the portfolio based on their weights.
    expected_returns (dictionary): Dictionary containing the expected returns for each portfolio
    """

    # Remove NaN values from closing prices or we may experience some issues
    closing_prices.dropna(inplace=True)

    portfolios = {}
    expected_returns = {}
    weightings = {}

    # Create the random portfolios, each containing the stocks' daily values based on their weights
    for i in range(num_portfolios):
        weights = random_weights(closing_prices.shape[1], min_weight, max_weight)
        weightings[i] = weights

        investment_per_stock = weights * capital
        # Calculate how many shares to buy (based on the closing price of the first day)
        num_shares = investment_per_stock / closing_prices.iloc[0]

        # Calculate the daily value of each stock in the portfolio
        portfolio = closing_prices * num_shares
        portfolios[i] = portfolio

        # Calculate the expected return of the portfolio
        # Each row in this dataframe is the total value of the portfolio on that day
        total_portfolio_value = portfolio.sum(axis=1)
        # Calculate the returns of the portfolio
        returns = total_portfolio_value.pct_change()

        # TODO: double check expected return calculation
        expected_return = returns.mean()
        expected_returns[i] = expected_return
    
    return portfolios, expected_returns, weightings

In [11]:
num_portfolios = 1000 # Number of random portfolios to generate

# TODO: remove the variable below after testing
final_closings = stock_data.iloc[:, :10] # Get first 10 stocks just for testing

rand_portfolios, expected_returns, weightings = random_portfolios(num_portfolios, final_closings)

# Pick the portfolio with the expected return closest to zero
# Here, we just use the min function to find the smallest absolute value in the dictionary
optimal_portfolio = min(expected_returns, key=lambda x: abs(expected_returns[x]))
print(f"The optimal portfolio has an expected return of about {float(expected_returns[optimal_portfolio]):.15%}.")

optimal_weights_df = pd.DataFrame(weightings[optimal_portfolio], index=final_closings.columns, columns=["Weighting"])
print(f"\nHere are the best weights for each stock:")
display(optimal_weights_df)

print("The optimal portfolio is:")
display(rand_portfolios[optimal_portfolio])

The optimal portfolio has an expected return of about 0.000099164916583%.

Here are the best weights for each stock:


,Weighting
AAPL,0.041981
ABBV,0.122140
ABT,0.112071
ACN,0.098791
AIG,0.151522
AMZN,0.086919
AXP,0.125139
BA,0.111411
BAC,0.123207
BIIB,0.026821


The optimal portfolio is:


,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB
Date,,,,,,,,,,
2023-01-03,31485.730602,91605.013305,84053.139528,74092.982658,113641.151407,65189.080906,93854.027454,83557.908102,92405.038903,20115.927135
2023-01-04,31810.481713,92344.034984,85303.425933,73840.756888,115320.577832,64672.551017,96035.788008,87085.994207,94142.289757,19981.638414
2023-01-05,31473.142591,92231.208758,84988.939228,72097.138642,114688.531703,63138.156644,93413.845900,87663.319998,93949.263053,20039.190401
2023-01-06,32631.168601,93957.474977,86162.521987,73805.115472,116566.607890,65386.579162,95799.752574,91088.768521,94886.825837,20604.381586
2023-01-09,32764.593074,91198.832006,86024.453083,75049.778989,115338.633144,66358.869354,95946.476514,89194.296327,93452.905666,20270.136918
...,...,...,...,...,...,...,...,...,...,...
2023-10-24,43662.630574,82539.283594,72723.834827,81174.388260,108115.296049,97654.487438,92131.585936,77985.670813,70234.449278,18601.864473
2023-10-25,43073.548504,81946.935148,71772.696738,80239.520602,110083.655683,92208.139356,91557.442008,76005.664038,70455.051976,18204.164906
2023-10-26,42013.700918,81913.088141,72087.189296,80064.066109,109885.012810,90825.663153,91442.607382,76587.263948,72026.852121,17788.019160


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.